In [ ]:
import torch, torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torch.utils
from alexnet import AlexNet
import torch.nn as nn
from torch.optim import Adam


In [ ]:
W = 227
transform = transforms.Compose([
    transforms.Resize((W, W)), # the original mnist image size is 28*28, alexnet input size is 227*227
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, ))
])

train = MNIST(root='./data', train=True, download=True, transform=transform)
test = MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size, shuffle=False)

model = AlexNet(10, 1)

In [ ]:
# img = torch.randn(batch_size, 1, W, W)
# output = model(img) 
# output.shape

In [ ]:
learning_rate = 0.01
num_epochs = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate)

In [ ]:
model = model.to(device)
for epoch in range(num_epochs):
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        print(labels.shape, outputs.shape)
        print(outputs[0], labels)
        break
        loss = criterion(outputs, labels)

        # backward and optimize 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("epoch:" + str(epoch) + "loss:" + str(loss))

